In [1]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

/home/aicontest/anaconda3/envs/construct/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

2025-02-25 09:11:29.664455: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-25 09:11:29.672280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740442289.681309  380848 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740442289.684001  380848 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 09:11:29.693385: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
sample = pd.read_csv("../data/sample_submission.csv")

In [44]:
# 공사종류,인적사고,물적사고,공종,사고객체,작업프로세스
grouped = train.groupby("작업프로세스")

In [45]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

  9%|▊         | 5/58 [00:01<00:17,  2.97it/s]


KeyboardInterrupt: 

In [39]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

Range 0.0 - 0.1: 1개
Range 0.1 - 0.2: 34개
Range 0.2 - 0.3: 203개
Range 0.3 - 0.4: 689개
Range 0.4 - 0.5: 1717개
Range 0.5 - 0.6: 3893개
Range 0.6 - 0.7: 6608개
Range 0.7 - 0.8: 7246개
Range 0.8 - 0.9: 2826개
Range 0.9 - 1.0: 179개


In [40]:
res

{'건축': '작업절차 및 상/하부 작업에 대한 안전교육 실시와 추락방지물 사전 제거 요청.',
 '건축 / 건축물': '안전교육 실시와 작업자 교육 철저를 통한 주의력 강화.',
 '건축 / 건축물 / 공동주택': '작업전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 철저를 통한 향후 조치 계획.',
 '건축 / 건축물 / 공장': '안전교육 실시와 현장 내 작업지시사항 철저 이행, 안전관리 교육 이행, 안전위험요소 제거 점검 및 대책 강구를 통한 재발 방지 대책.',
 '건축 / 건축물 / 관광 휴게시설': '작업 전 안전교육 및 작업 중 안전관리 철저에 대한 재발 방지 대책과 향후 조치 계획.',
 '건축 / 건축물 / 교육연구시설': '작업 시작 전 안전교육 실시 및 법정 안전교육 강화, 현장 위험 요인 수시 점검, 위험 부분 정리정돈 철저, 사전 안내 시설 설치를 통한 작업자의 안전의식 고취.',
 '건축 / 건축물 / 교정 및 군사시설': '작업전 안전교육 실시, 안전시설 재점검, 안전장구류 착용 철저 등의 재발 방지 대책과 향후 조치 계획.',
 '건축 / 건축물 / 근린생활시설': '작업자 안전교육 강화와 안전관리를 위한 현장감독 철저를 통한 재발 방지 대책 및 안전교육 실시와 재발 방지 철저의 향후 조치 계획.',
 '건축 / 건축물 / 기타': '작업자 작업순서 숙지 교육 및 자재 안전점검과 작업교육 및 작업자재 안전관리 철저를 통한 재발 방지 대책.',
 '건축 / 건축물 / 노유자시설': '작업자 안전교육 강화, 작업방법 안내 교육 실시, 공사현장 관리 철저 및 안전지도 강화를 통한 재발 방지 대책 마련.',
 '건축 / 건축물 / 단독주택': '현장정리 철저와 작업자 안전교육을 통한 재발 방지 대책 및 향후 조치 계획.',
 '건축 / 건축물 / 동물 및 식물 관련시설': '재해원인 재발방지를 위한 현장 여건 보강, 안전교육 강화 및 공사장 안전관리 철저 요청과 근로자 안전교육 철저 조치.',
 '건축 / 건

In [8]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [ ]:
sample.head(3)

In [10]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Columns: 770 entries, ID to vec_767
dtypes: float64(768), object(2)
memory usage: 5.7+ MB


In [11]:
for i in range(len(test)):
    accident = test.loc[i, "인적사고"]
    sample.loc[i, "재발방지대책 및 향후조치계획"] = res[accident]
    sample.iloc[i, 2:] = res_v[accident]

In [12]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Columns: 770 entries, ID to vec_767
dtypes: float64(768), object(2)
memory usage: 5.7+ MB


In [13]:
sample.to_csv("baseline.csv", index=False, encoding='utf-8-sig')